# Creation of hydrocerussite calibrant

In this tutorial we will see how to create a new calibrant. For this example we will use one of the componant of mostpaintings: hydrocerussite.

The cell parameter are definied in this document:
http://rruff.geo.arizona.edu/AMS/AMC_text_files/11987_amc.txt

The first step is to record the cell parameters and provide them to pyFAI to define the cell.

In [1]:
import pyFAI
print("pyFAI version",pyFAI.version)

pyFAI version 2025.11.0-dev0


In [2]:
from pyFAI.calibrant import Cell
hydroc_hex = Cell.hexagonal(5.24656, 23.7023)

#This is an alternative representation, where extinction rules are already definied ... but that's cheating
hydroc_rho = Cell.hexagonal(5.24656, 23.7023, lattice_type="R")

Chromium oxide has a crystal structure de Corrundom which is R-3m (space group 166). 
The selection rules are rather complicated and are available in:
![Space-group 166](http://img.chem.ucl.ac.uk/sgp/large/166bz2.gif "Space-group 166")


We will setup a function corresponding to the selection rules. It returns True if the reflection is active and False  otherwise.


In [3]:
from pyFAI.crystallography.space_groups import ReflectionCondition
#from http://img.chem.ucl.ac.uk/sgp/large/166bz2.htm
reflection_condition_166 = ReflectionCondition.group166_R3bar_m
import inspect
print(inspect.getsource(reflection_condition_166))

    @staticmethod
    def group166_R3bar_m(h: int, k: int, l: int) -> bool:  # noqa: E741
        """
        Space group 166: R3̅m. Trigonal (hexagonal axes), rhombohedral lattice.
        Valid reflections must satisfy:
        - hkil:                          -h + k + l = 3n
        - hki0 (l = 0):                  -h + k = 3n
        - hh(-2h)l:                      l = 3n
        - h(-h)0l (i = 0, k = -h):       h + l = 3n
        - 000l (h = k = 0):              l = 3n
        - h(-h)00 (l = 0, k = -h):       h = 3n

        Source:
            Reflection conditions from ITC (in hkil), adapted to (h, k, l)
            using the relation i = -(h + k).
            JKC: http://img.chem.ucl.ac.uk/sgp/large/166bz2.htm
        validated
        """
        if (-h + k + l) % 3 != 0:
            return False  # hkil
        if l == 0:
            return (-h + k) % 3 == 0  # hki0
        if h == k:
            return l % 3 == 0  # hh(-2h)l
        if k == -h:
            return (h + l) % 

In [4]:
# Use the actual selection rule, not the short version:
#cro.selection_rules.append(lambda h, k, l: ((-h + k + l) % 3 == 0))
hydroc_hex.selection_rules.append(reflection_condition_166)

In [5]:
reflections = hydroc_hex.calculate_dspacing(1)
d_spacing = list(reflections.keys())
d_spacing.sort(reverse=True)

print("+" + "-"*9 + "+" + "-"*60 + "+")
print("|" + "d-spacing" + "|" + "Miller indices"+ " "*46 + "|")
print("+" + "-"*9 + "+" + "-"*60 + "+")
for d in d_spacing:
    print(f"|{d:9f}|{' '.join([str(i) for i in reflections[d]])[:60]:60s}|")
print("+" + "-"*9 + "+" + "-"*60 + "+")

+---------+------------------------------------------------------------+
|d-spacing|Miller indices                                              |
+---------+------------------------------------------------------------+
| 7.900767|(0 0 -3) (0 0 3)                                            |
| 4.462403|(1 -1 -1) (-1 0 -1) (0 1 -1) (0 -1 1) (1 0 1) (-1 1 1)      |
| 4.242534|(0 -1 -2) (1 0 -2) (-1 1 -2) (1 -1 2) (-1 0 2) (0 1 2)      |
| 3.950383|(0 0 -6) (0 0 6)                                            |
| 3.605663|(1 -1 -4) (-1 0 -4) (0 1 -4) (0 -1 4) (1 0 4) (-1 1 4)      |
| 3.280218|(0 -1 -5) (1 0 -5) (-1 1 -5) (1 -1 5) (-1 0 5) (0 1 5)      |
| 2.715045|(1 -1 -7) (-1 0 -7) (0 1 -7) (0 -1 7) (1 0 7) (-1 1 7)      |
| 2.633589|(0 0 -9) (0 0 9)                                            |
| 2.623280|(1 -2 0) (-1 -1 0) (2 -1 0) (-2 1 0) (1 1 0) (-1 2 0)       |
| 2.489635|(1 -2 -3) (-1 -1 -3) (2 -1 -3) (-2 1 -3) (1 1 -3) (-1 2 -3) |
| 2.481778|(0 -1 -8) (1 0 -8) (-1 1 -8) (1 -1 8) (-

In [6]:
print("Hexagonal vs Rhombohedral:")
print("length is the same: ", len(hydroc_hex.calculate_dspacing(1)) == len(hydroc_rho.calculate_dspacing(1)))
ds_hex=list(hydroc_hex.calculate_dspacing(1).keys())
ds_hex.sort()
ds_rho=list(hydroc_rho.calculate_dspacing(1).keys())
ds_rho.sort()
print("Content is the same:", ds_hex == ds_rho)

Hexagonal vs Rhombohedral:
length is the same:  True
Content is the same: True


In [7]:
hydroc_rho.save("hydrocerussite", "basic lead carbonate (R-3m)", dmin=1, doi="https://doi.org/10.1107/S0108270102006844")

## Conclusion
This is an advanced tutorial, most user won't have to define their own calibrant. You can also contact the developers to get your own calibrant integrated into pyFAI which makes things easier for you and other users.